In [ ]:
from robomaster import robot
import time
import csv
import os

# เก็บข้อมูล log
log_data_pos = []

# พารามิเตอร์ของ PD controller
Kp = 5.0                  # ค่า Proportional gain
Kd = 2.0                  # ค่า Derivative gain
setpoint = 2.0            # ระยะเป้าหมาย (เมตร)
current_x = 0.0           # ตำแหน่ง x ปัจจุบัน
prev_error = 0.0
prev_time = None

# เขียน CSV
def write_csv_once(file_path, fieldnames, entry):
    file_exists = os.path.isfile(file_path)
    with open(file_path, mode="a", newline="") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        if not file_exists or os.path.getsize(file_path) == 0:
            writer.writeheader()
        writer.writerow(entry)

# เก็บตำแหน่ง
def pos_info_handler(position_info):
    global current_x
    timestamp = time.time()
    current_x = position_info[0]  # อัปเดตตำแหน่ง x ปัจจุบัน

    entry = {"timestamp": timestamp, "x": position_info[0], "y": position_info[1], "z": position_info[2]}
    log_data_pos.append(entry)
    print(f"Position X: {position_info[0]:.2f}")
    write_csv_once("log_poshigh1.csv", ["timestamp", "x", "y", "z"], entry)

if __name__ == '__main__':
    ep_robot = robot.Robot()
    ep_robot.initialize(conn_type="ap")

    ep_chassis = ep_robot.chassis
    ep_chassis.sub_position(freq=10, callback=pos_info_handler)

    print("Starting PD-Control movement...")

    try:
        while True:
            current_time = time.time()
            error = setpoint - current_x

            # เวลาระหว่างรอบ (dt)
            if prev_time is None:
                dt = 0.1
            else:
                dt = current_time - prev_time

            derivative = (error - prev_error) / dt if dt > 0 else 0.0

            # PD Control
            speed = (Kp * error) + (Kd * derivative)

            # จำกัดช่วงความเร็ว
            speed = max(min(speed, 1.0), -1.0)

            if abs(error) <= 0.01:
                ep_chassis.drive_speed(x=0, y=0, z=0)
                print("Target distance reached.")
                break
            # สั่งการเคลื่อนที่
            ep_chassis.drive_speed(x=speed, y=0, z=0)

            # Debug info
            print(f"Error: {error:.2f}, Derivative: {derivative:.2f}, Speed: {speed:.2f}")

            # เตรียมรอบถัดไป
            prev_error = error
            prev_time = current_time

            time.sleep(0.1)

    finally:
        ep_chassis.unsub_position()
        ep_robot.close()
        print("Robot closed.")


Starting PD-Control movement...
Error: 2.00, Derivative: 20.00, Speed: 1.00
Position X: 0.00
Error: 2.00, Derivative: 0.00, Speed: 1.00
Position X: 0.00
Error: 2.00, Derivative: -0.04, Speed: 1.00
Position X: 0.04
Error: 1.96, Derivative: -0.37, Speed: 1.00
Position X: 0.13
Error: 1.87, Derivative: -0.78, Speed: 1.00
Position X: 0.23
Error: 1.77, Derivative: -0.91, Speed: 1.00
Position X: 0.32
Error: 1.68, Derivative: -0.85, Speed: 1.00
Position X: 0.40
Error: 1.60, Derivative: -0.67, Speed: 1.00
Position X: 0.47
Error: 1.53, Derivative: -0.63, Speed: 1.00
Position X: 0.55
Error: 1.45, Derivative: -0.73, Speed: 1.00
Position X: 0.63
Position X: 0.72Error: 1.37, Derivative: -0.81, Speed: 1.00

Position X: 0.80
Error: 1.20, Derivative: -1.51, Speed: 1.00
Position X: 0.87
Error: 1.13, Derivative: -0.69, Speed: 1.00
Position X: 1.00
Error: 1.00, Derivative: -1.13, Speed: 1.00
Position X: 1.08
Error: 0.92, Derivative: -0.76, Speed: 1.00
Position X: 1.16
Error: 0.84, Derivative: -0.72, Speed

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# อ่านไฟล์ CSV ทั้งสอง
df_high = pd.read_csv("log_poshigh.csv")
df_low = pd.read_csv("log_poslow.csv")

# ดึงข้อมูล timestamp และตำแหน่ง x
time_high = df_high['timestamp']
x_high = df_high['x']

time_low = df_low['timestamp']
x_low = df_low['x']

# สร้างกราฟ subplot แบบแนวตั้ง (2 แถว 1 คอลัมน์)
plt.figure(figsize=(10, 12))

# กราฟ High (แกน X = Time, แกน Y = Position)
plt.subplot(2, 1, 1)
plt.plot(time_high, x_high, color='red')
plt.xlabel("Time (s)")
plt.ylabel("Position X (m)")
plt.title("Position X over Time - High")
plt.grid(True)

# กราฟ Low (แกน X = Time, แกน Y = Position)
plt.subplot(2, 1, 2)
plt.plot(time_low, x_low, color='blue')
plt.xlabel("Time (s)")
plt.ylabel("Position X (m)")
plt.title("Position X over Time - Low")
plt.grid(True)

# จัด layout ให้ดูดี
plt.tight_layout()

# แสดงผล
plt.show()


In [7]:
from robomaster import robot
import time
import csv
import os

# PID parameters
Kp = 0.5
Ki = 0.0
Kd = 0.0

# กำหนดตัวแปรสถานะ
current_x = 0.0

# ---------- CSV Logger ----------
def write_csv_once(file_path, fieldnames, entry):
    file_exists = os.path.isfile(file_path)
    with open(file_path, mode="a", newline="") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        if not file_exists or os.path.getsize(file_path) == 0:
            writer.writeheader()
        writer.writerow(entry)

# ---------- Callback อัปเดตตำแหน่ง ----------
def pos_info_handler(position_info):
    global current_x
    current_x = position_info[0]
    entry = {"timestamp": time.time(), "x": position_info[0], "y": position_info[1], "z": position_info[2]}
    write_csv_once("log_pos.csv", ["timestamp", "x", "y", "z"], entry)

# ---------- PID ควบคุมการเดินหน้า ----------
def move_forward_pid(ep_chassis, distance):
    global current_x

    # รอรับตำแหน่งก่อนเริ่ม
    while current_x == 0.0:
        print("Waiting for position info...")
        time.sleep(0.1)

    start_x = current_x
    target_x = start_x + distance

    prev_error = 0.0
    integral = 0.0
    prev_time = time.time()

    while True:
        current_time = time.time()
        dt = current_time - prev_time if current_time > prev_time else 0.1

        error = target_x - current_x
        integral += error * dt
        derivative = (error - prev_error) / dt if dt > 0 else 0.0

        speed = (Kp * error) + (Ki * integral) + (Kd * derivative)
        #speed = max(min(speed, 0.2), -0.2)  # จำกัดความเร็ว

        if abs(error) <= 0.03:
            ep_chassis.drive_speed(x=0, y=0, z=0)
            time.sleep(0.2)
            print("✅ Target reached.")
            break

        ep_chassis.drive_speed(x=speed, y=0, z=0)
        print(f"Current X: {current_x:.2f} | Target: {target_x:.2f} | Speed: {speed:.2f}")

        prev_error = error
        prev_time = current_time
        time.sleep(0.1)

# ---------- Main ----------
if __name__ == '__main__':
    ep_robot = robot.Robot()
    ep_robot.initialize(conn_type="ap")

    ep_chassis = ep_robot.chassis
    ep_chassis.sub_position(freq=10, callback=pos_info_handler)

    try:
        for i in range(4):
            move_forward_pid(ep_chassis, distance=0.6)
            ep_chassis.move(x=0, y=0, z=-90, z_speed=30).wait_for_completed()

        print("✅ Completed square path.")

    finally:
        ep_chassis.unsub_position()
        ep_robot.close()


Waiting for position info...
Waiting for position info...
Current X: 0.00 | Target: 0.60 | Speed: 0.30
Current X: 0.00 | Target: 0.60 | Speed: 0.30
Current X: 0.00 | Target: 0.60 | Speed: 0.30
Current X: 0.03 | Target: 0.60 | Speed: 0.28
Current X: 0.08 | Target: 0.60 | Speed: 0.26
Current X: 0.11 | Target: 0.60 | Speed: 0.24
Current X: 0.13 | Target: 0.60 | Speed: 0.23
Current X: 0.14 | Target: 0.60 | Speed: 0.23
Current X: 0.16 | Target: 0.60 | Speed: 0.22
Current X: 0.19 | Target: 0.60 | Speed: 0.20
Current X: 0.22 | Target: 0.60 | Speed: 0.19
Current X: 0.26 | Target: 0.60 | Speed: 0.17
Current X: 0.28 | Target: 0.60 | Speed: 0.16
Current X: 0.29 | Target: 0.60 | Speed: 0.15
Current X: 0.31 | Target: 0.60 | Speed: 0.15
Current X: 0.32 | Target: 0.60 | Speed: 0.14
Current X: 0.33 | Target: 0.60 | Speed: 0.13
Current X: 0.35 | Target: 0.60 | Speed: 0.13
Current X: 0.36 | Target: 0.60 | Speed: 0.12
Current X: 0.37 | Target: 0.60 | Speed: 0.11
Current X: 0.38 | Target: 0.60 | Speed: 0.

2025-08-08 11:29:38,111 ERROR client.py:163 Client: send_sync_msg wait msg receiver:0900, cmdset:0x48, cmdid:0x04 timeout!
2025-08-08 11:29:41,114 ERROR client.py:163 Client: send_sync_msg wait msg receiver:0900, cmdset:0x3f, cmdid:0xd1 timeout!


OSError: [WinError 10049] The requested address is not valid in its context

In [13]:
from robomaster import robot
import time

# PID Controller
class PID:
    def __init__(self, Kp, Ki, Kd):
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        self.prev_error = 0.0
        self.integral = 0.0

    def compute(self, error, dt):
        self.integral += error * dt
        derivative = (error - self.prev_error) / dt if dt > 0 else 0.0
        output = self.Kp * error + self.Ki * self.integral + self.Kd * derivative
        self.prev_error = error
        return output

# ค่า PID และระยะ
Kp, Ki, Kd = 1.0, 0.0, 0.1
step_distance = 0.6
current_position = {"x": 0.0, "y": 0.0, "z": 0.0}

# callback ตำแหน่ง
def on_position_change(position_info):
    current_position["x"] = position_info['x']
    current_position["y"] = position_info['y']
    current_position["z"] = position_info['z']

# เริ่มเชื่อมต่อ
ep_robot = robot.Robot()
ep_robot.initialize(conn_type="ap")

ep_chassis = ep_robot.chassis

# subscribe การติดตามตำแหน่งแบบต่อเนื่อง
ep_chassis.sub_position(freq=10, callback=on_position_change)
time.sleep(0.5)  # รอให้เริ่มรับค่าตำแหน่ง

pid_x = PID(Kp, Ki, Kd)

for i in range(4):
    print(f"\n▶️ STEP {i+1}: Move forward {step_distance} meters")
    
    start_time = time.time()
    start_x = current_position["x"]
    target_x = start_x + step_distance

    while True:
        curr_x = current_position["x"]
        error = target_x - curr_x

        if abs(error) < 0.01:
            ep_chassis.drive_speed(x=0.0, y=0.0, z=0.0)
            break

        dt = 0.05
        speed = pid_x.compute(error, dt)

        # จำกัดความเร็ว
        max_speed = 0.2
        speed = max(min(speed, max_speed), -max_speed)

        ep_chassis.drive_speed(x=speed, y=0.0, z=0.0)
        print(f"Current X: {curr_x:.2f} | Target: {target_x:.2f} | Speed: {speed:.2f}")
        time.sleep(dt)

    # หยุดก่อนหมุน
    ep_chassis.drive_speed(x=0.0, y=0.0, z=0.0)
    time.sleep(0.2)

    print("↪️ Rotating 90 degrees...")
    ep_chassis.move(z=90).wait_for_completed()

    # รีเซ็ต PID
    pid_x.prev_error = 0.0
    pid_x.integral = 0.0
    time.sleep(0.5)

# ปิดการติดตามตำแหน่ง
ep_chassis.unsub_position()

# ปิดการเชื่อมต่อ
ep_robot.close()



▶️ STEP 1: Move forward 0.6 meters
Current X: 0.00 | Target: 0.60 | Speed: 0.20
Current X: 0.00 | Target: 0.60 | Speed: 0.20
Current X: 0.00 | Target: 0.60 | Speed: 0.20
Current X: 0.00 | Target: 0.60 | Speed: 0.20
Current X: 0.00 | Target: 0.60 | Speed: 0.20
Current X: 0.00 | Target: 0.60 | Speed: 0.20
Current X: 0.00 | Target: 0.60 | Speed: 0.20
Current X: 0.00 | Target: 0.60 | Speed: 0.20
Current X: 0.00 | Target: 0.60 | Speed: 0.20
Current X: 0.00 | Target: 0.60 | Speed: 0.20
Current X: 0.00 | Target: 0.60 | Speed: 0.20
Current X: 0.00 | Target: 0.60 | Speed: 0.20
Current X: 0.00 | Target: 0.60 | Speed: 0.20
Current X: 0.00 | Target: 0.60 | Speed: 0.20
Current X: 0.00 | Target: 0.60 | Speed: 0.20
Current X: 0.00 | Target: 0.60 | Speed: 0.20
Current X: 0.00 | Target: 0.60 | Speed: 0.20
Current X: 0.00 | Target: 0.60 | Speed: 0.20
Current X: 0.00 | Target: 0.60 | Speed: 0.20
Current X: 0.00 | Target: 0.60 | Speed: 0.20
Current X: 0.00 | Target: 0.60 | Speed: 0.20
Current X: 0.00 | T

KeyboardInterrupt: 

# Normal

In [25]:
from robomaster import robot
import time
import csv
import os

# -----------------------------
# Target distance (meters)
# -----------------------------
target_distance = 1.2

# Position logging
log_data_pos = []
current_x = 0.0
current_y = 0.0
current_speed = 0.0  # ✅ เพิ่มตัวแปรนี้ไว้เก็บ speed ปัจจุบัน

# -----------------------------
# CSV Logging Function
# -----------------------------
def write_csv_once(file_path, fieldnames, entry):
    file_exists = os.path.isfile(file_path)
    with open(file_path, mode="a", newline="") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        if not file_exists or os.path.getsize(file_path) == 0:
            writer.writeheader()
        writer.writerow(entry)

# -----------------------------
# Position Callback
# -----------------------------
def pos_info_handler(position_info):
    global current_x, current_y, current_speed
    timestamp = time.time()
    current_x = position_info[0]
    current_y = position_info[1]

    entry = {
        "timestamp": timestamp,
        "x": position_info[0],
        "y": position_info[1],
        "z": position_info[2],
        "speed": current_speed  # ✅ เพิ่มค่า speed ที่บันทึกลงไป
    }
    log_data_pos.append(entry)
    write_csv_once("log_square_simple.csv", ["timestamp", "x", "y", "z", "speed"], entry)

# -----------------------------
# Move Straight without PI
# -----------------------------
def move_straight(ep_chassis, axis='x', distance=1.2, speed=0.3):
    """Move in straight line on specified axis at fixed speed until target distance reached"""
    global current_x, current_y, current_speed
    current_speed = abs(speed)  # ✅ อัปเดตค่าความเร็วไว้ให้ pos_info_handler อ่านได้

    print(f"Moving {distance} meters on {axis.upper()} axis...")

    if axis == 'x':
        start_pos = current_x
        while abs(current_x - start_pos) < abs(distance):
            ep_chassis.drive_speed(x=speed if distance > 0 else -speed, y=0, z=0)
            time.sleep(0.05)
    elif axis == 'y':
        start_pos = current_y
        while abs(current_y - start_pos) < abs(distance):
            ep_chassis.drive_speed(x=0, y=speed if distance > 0 else -speed, z=0)
            time.sleep(0.05)

    ep_chassis.drive_speed(x=0, y=0, z=0)
    current_speed = 0.0  # ✅ หยุดรถแล้วก็ให้ speed เป็น 0
    print(f"Reached target on {axis.upper()} axis.")

# -----------------------------
# Main Program
# -----------------------------
if __name__ == '__main__':
    ep_robot = robot.Robot()
    ep_robot.initialize(conn_type="ap")

    ep_chassis = ep_robot.chassis
    ep_chassis.sub_position(freq=10, callback=pos_info_handler)

    try:
        print("== Start Square Movement ==")
        move_straight(ep_chassis, axis='x', distance=target_distance)
        move_straight(ep_chassis, axis='y', distance=target_distance)
        move_straight(ep_chassis, axis='x', distance=-target_distance)
        move_straight(ep_chassis, axis='y', distance=-target_distance)

    finally:
        ep_chassis.unsub_position()
        ep_robot.close()
        print("Robot closed.")


== Start Square Movement ==
Moving 1.2 meters on X axis...
Reached target on X axis.
Moving 1.2 meters on Y axis...
Reached target on Y axis.
Moving -1.2 meters on X axis...
Reached target on X axis.
Moving -1.2 meters on Y axis...
Reached target on Y axis.
Robot closed.


# PI

In [ ]:
from robomaster import robot
import time
import csv
import os

# -----------------------------
# PI Controller Parameters
# -----------------------------
Kp = 1.5   # Proportional gain
Ki = 0.3   # Integral gain
target_distance = 1.2  # meters

# Global state
log_data_pos = []
current_x = 0.0
current_y = 0.0
current_speed = 0.0  # เก็บค่าความเร็วที่ส่งไปใน drive_speed()

# -----------------------------
# CSV Logging Function
# -----------------------------
def write_csv_once(file_path, fieldnames, entry):
    file_exists = os.path.isfile(file_path)
    with open(file_path, mode="a", newline="") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        if not file_exists or os.path.getsize(file_path) == 0:
            writer.writeheader()
        writer.writerow(entry)

# -----------------------------
# Position Callback
# -----------------------------
def pos_info_handler(position_info):
    global current_x, current_y, current_speed
    timestamp = time.time()
    current_x = position_info[0]
    current_y = position_info[1]

    entry = {
        "timestamp": timestamp,
        "x": position_info[0],
        "y": position_info[1],
        "z": position_info[2],
        "speed": current_speed
    }
    log_data_pos.append(entry)
    write_csv_once("logPI_slide_square.csv", ["timestamp", "x", "y", "z", "speed"], entry)

# -----------------------------
# PI Controlled Straight Movement
# -----------------------------
def move_straight_pi(ep_chassis, axis='x', distance=1.2):
    global current_x, current_y, current_speed
    print(f"Moving {distance} meters on {axis.upper()} axis...")

    if axis == 'x':
        start_pos = current_x
        get_position = lambda: current_x
    elif axis == 'y':
        start_pos = current_y
        get_position = lambda: current_y
    else:
        raise ValueError("Axis must be 'x' or 'y'")

    # PI Controller variables
    error_sum = 0.0
    last_time = time.time()

    while True:
        now = time.time()
        dt = now - last_time
        last_time = now

        # Error = Target distance - Distance traveled
        current_pos = get_position()
        traveled = current_pos - start_pos
        error = distance - traveled

        # Integral term
        error_sum += error * dt

        # PI output
        output = (Kp * error) + (Ki * error_sum)

        # Limit output speed
        output = max(min(output, 0.5), -0.5)
        current_speed = output  # update speed globally

        # Stop condition
        if abs(error) <= 0.01:
            ep_chassis.drive_speed(x=0, y=0, z=0)
            current_speed = 0.0  # stop speed
            print(f"Reached target on {axis.upper()} axis.")
            break

        # Apply movement
        if axis == 'x':
            ep_chassis.drive_speed(x=output, y=0, z=0)
        else:
            ep_chassis.drive_speed(x=0, y=output, z=0)

        # Debug print
        print(f"Error: {error:.3f}, Integral: {error_sum:.3f}, Output: {output:.3f}")
        time.sleep(0.05)

# -----------------------------
# Main Program
# -----------------------------
if __name__ == '__main__':
    ep_robot = robot.Robot()
    ep_robot.initialize(conn_type="ap")

    ep_chassis = ep_robot.chassis
    ep_chassis.sub_position(freq=10, callback=pos_info_handler)

    try:
        print("== Start Square Movement with PI-Control ==")
        move_straight_pi(ep_chassis, axis='x', distance=target_distance)
        move_straight_pi(ep_chassis, axis='y', distance=target_distance)
        move_straight_pi(ep_chassis, axis='x', distance=-target_distance)
        move_straight_pi(ep_chassis, axis='y', distance=-target_distance)

    finally:
        ep_chassis.unsub_position()
        ep_robot.close()
        print("Robot closed.")


== Start Square Movement with PI-Control ==
Moving 1.2 meters on X axis...
Error: 1.200, Integral: 0.000, Output: 0.500
Error: 1.200, Integral: 0.074, Output: 0.500
Error: 1.200, Integral: 0.153, Output: 0.500
Error: 1.200, Integral: 0.232, Output: 0.500
Error: 1.189, Integral: 0.294, Output: 0.500
Error: 1.189, Integral: 0.358, Output: 0.500
Error: 1.140, Integral: 0.428, Output: 0.500
Error: 1.077, Integral: 0.498, Output: 0.500
Error: 1.077, Integral: 0.553, Output: 0.500
Error: 1.023, Integral: 0.605, Output: 0.500
Error: 1.023, Integral: 0.658, Output: 0.500
Error: 0.986, Integral: 0.720, Output: 0.500
Error: 0.986, Integral: 0.771, Output: 0.500
Error: 0.953, Integral: 0.819, Output: 0.500
Error: 0.953, Integral: 0.881, Output: 0.500
Error: 0.909, Integral: 0.931, Output: 0.500
Error: 0.858, Integral: 0.983, Output: 0.500
Error: 0.858, Integral: 1.031, Output: 0.500
Error: 0.810, Integral: 1.080, Output: 0.500
Error: 0.810, Integral: 1.126, Output: 0.500
Error: 0.770, Integral: 1